# Homework 4 - Spark

In this homework, we are practicing Apache Spark.

You are required to turn in this notebook as BDM\_HW4\_Spark\_**NetId**.ipynb. You will be asked to complete each task using Apache Spark. Output can be printed in the notebook.

## Task 1 (5 points)

You are asked to implement Homework 3 using Spark. The description is provided below for your convenience.

You are asked to implement the Social Triangle example discussed in class. In particular, given the email dataset, please list all "reciprocal" relationships in the company. Recall that:

If A emails B and B emails A, then A and B is *reciprocal*.

If A emails B but B doesn’t email A, then A and B is *directed*.

**Dataset:** We will use a subset of the open [Enron Email Dataset](https://www.cs.cmu.edu/~./enron/ "Enron Email Dataset"), which contains approximately 10,000 simplified email headers from the Enron Corporation. You can download this dataset from NYU Classes as **enron_mails_small.csv**. The file contains 3 columns *Date*, *From*, and *To*. Their description is as follows:

|Column name|Description|
|--|--|
|Date |The date and time of the email, in the format YYYY-MM-DD hh-mm-ss, <br />e.g. "1998-10-30 07:43:00" |
|From |The sender email address, <br />e.g. "mark.taylor@enron.com" |
|To | A list of recipients' email addresses separated by semicolons ';', <br />e.g. "jennifer.fraser@enron.com;jeffrey.hodge@enron.com" |

Note that, we only care about users employed by Enron, or only relationships having email addresses that end with *'@enron.com'*.

The expected output is also provided below. For each reciprocal relationship, please output a tuple consisting of two strings. The first one is always **'reciprocal'**. And the second one is a string showing the name of the two person in the following format: **'Jane Doe : John Doe'**. The names should be presented in the lexical order, i.e. there will not be a 'John Doe : Jane Doe' since 'Jane' is ordered before 'John.

Though the dataset only contains email addresses, not actual names, we're assuming that the email aliases were created based on their name. For example:

|Email Address|Converted Name|
|--|--|
|mark.taylor@enron.com|Mark Taylor|
|alan.aronowitz@enron.com|Alan Aronowitz|
|marc.r.cutler@enron.com|Marc R Cutler|
|hugh@enron.com|Hugh|

Please fill the code block with a series of MapReduce jobs using your own mapper and reducer functions. Be sure to include the naming convention logic into one of your mappers and/or reducers.

In [1]:
import pyspark
from pyspark.sql import Row
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import FloatType
from pyspark.sql.functions import udf

In [2]:
#initialize session / spark container
sc = SparkSession \
    .builder \
    .appName("Enron Emails") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [3]:
#test container
sc

In [217]:
#read in the email file csv 
enron = 'enron_mails_small.csv'
enron_df = sc.read.csv(enron)

In [218]:
#initial filtering
enron_df = enron_df.select("_c1","_c2")
enron_df = enron_df.select("_c1",explode(split(enron_df["_c2"],";")).alias("to"))
enron_df = enron_df.filter("_c1 like '%@enron.com%'") #filter out non-enron emails

#clean up the dataframe
enron_df = enron_df.filter("to like '%@enron.com%'") #filter out non-enron emails
enron_df = enron_df.filter("_c1 like '%@enron.com%'") #filter out non-enron emails
enron_df = enron_df.withColumn('_c1', regexp_replace('_c1', '@enron.com', '')) #remove email strings
enron_df = enron_df.withColumn('to', regexp_replace('to', '@enron.com', '')) #remove email strings

In [222]:
#remove special characters
period_remove = udf(lambda x : x.replace('.', ' '))
enron_df = enron_df.withColumn('_c1',period_remove(enron_df._c1))
enron_df = enron_df.withColumn('to',period_remove(enron_df.to))

In [223]:
#create a dataframe with c1 is the from person and c2 is a set of everyone they have sent an email to
enron_df_from = enron_df.groupby("_c1").agg(collect_set("to"))

#create a dataframe where _c2 is the person receiving the email and the set is all the people they've received from
enron_df_to = enron_df.groupby("to").agg(collect_set("_c1"))

#join dataframes 
enron_df = enron_df_from.join(enron_df_to,enron_df_from._c1 == enron_df_to.to)

#renaming columns
enron_df = enron_df.select(col("_c1").alias("from"), col("collect_set(to)").alias("to_list"),col("to").alias("to"),
                          col("collect_set(_c1)").alias("from_list"))

In [227]:
mergeCols = udf(lambda x,y: list(set(x) & set(y)),returnType='array<string>')
enron_df = enron_df.withColumn("to_from_list", mergeCols(col("to_list"), col("from_list")))

In [228]:
enron_df = enron_df.select(col("from"),col("to_from_list"))

In [229]:
filterdf = enron_df.filter(size('to_from_list')>0)

In [230]:
filterdf = filterdf.withColumn("to_from_string", filterdf["to_from_list"].cast('String'))
filterdf = filterdf.select("from",explode(split(filterdf["to_from_string"],",")).alias("to"))

In [232]:
char_remove = udf(lambda x : x.replace('[', ''))
filterdf = filterdf.withColumn('to',char_remove(filterdf.to))

char_remove = udf(lambda x : x.replace(']', ''))
filterdf = filterdf.withColumn('to',char_remove(filterdf.to))

In [234]:
filterdf = filterdf.withColumn('from', ltrim(filterdf['from']))
filterdf = filterdf.withColumn('to', ltrim(filterdf['to']))

In [235]:
sort_list = udf(lambda x,y: sorted((x+','+y).split(',')),returnType='array<string>')
filterdf = filterdf.withColumn("sorted_list", sort_list(col("to"), col("from")))

In [236]:
filterdf = filterdf.select(filterdf.sorted_list[0],filterdf.sorted_list[1])
filterdf = filterdf.select(["sorted_list[0]",'sorted_list[1]']).distinct()

In [238]:
dup_remove = udf(lambda x,y: 1 if x==y else 0)
filterdf = filterdf.withColumn('same_flag',dup_remove(filterdf['sorted_list[0]'],filterdf['sorted_list[1]']))

In [239]:
filterdf = filterdf.filter("same_flag = 0")
filterdf = filterdf.withColumn('reciprocal',concat(col('sorted_list[0]'),lit(" : "),col('sorted_list[1]')))
filterdf = filterdf.select(['reciprocal'])

In [241]:
filterdf.head(35)

[Row(reciprocal='janette elbertson : richard sanders'),
 Row(reciprocal='carol clair : richard sanders'),
 Row(reciprocal='mark taylor : tana jones'),
 Row(reciprocal='mark haedicke : richard sanders'),
 Row(reciprocal='carol clair : sara shackleton'),
 Row(reciprocal='pinnamaneni krishnarao : vince kaminski'),
 Row(reciprocal='debra perlingiere : kevin ruscitti'),
 Row(reciprocal='drew fossum : susan scott'),
 Row(reciprocal='grant masson : vince kaminski'),
 Row(reciprocal='carol clair : debra perlingiere'),
 Row(reciprocal='elizabeth sager : mark taylor'),
 Row(reciprocal='carol clair : mark taylor'),
 Row(reciprocal='michelle cash : twanda sweet'),
 Row(reciprocal='gerald nemec : susan scott'),
 Row(reciprocal='mark haedicke : twanda sweet'),
 Row(reciprocal='rosalee fleming : steven kean'),
 Row(reciprocal='elizabeth sager : richard sanders'),
 Row(reciprocal='brenda whitehead : elizabeth sager'),
 Row(reciprocal='janette elbertson : mark taylor'),
 Row(reciprocal='eric bass : sus

## Task 2 (5 points)

You are asked to implement Task 2 of Lab 5. The description is provided below for your convenience.

We’ll be using two NYC open data sets: the SAT Results and the NYC High School Directory data sets. Both can be downloaded from the links below, or from online class resources.

**Dataset**: *Please note that each school is uniquely identified by an DBN code, which should be found on both data sets.*

**SAT_Results.csv**
Source: https://nycopendata.socrata.com/Education/SAT-Results/f9bf-2cp4  
Description: “The most recent school level results for New York City on the SAT. Results are available at the school level for the graduating seniors of 2012.”

**DOE_High_School_Directory_2014-2015.csv**
Source: https://data.cityofnewyork.us/Education/DOE-High-School-Directory-2014-2015/n3p6-zve2  
Description: “Directory of NYC High Schools.”

We would like to know how the Math scores vary across bus lines or subway lines serving the schools. Your task is to compute the average Math scores of all schools along each bus line and subway line. You can find the bus and subway lines serving each school in the High School Dictionary as bus and subway columns.

The expected results are two lists:
1. A list of key/value pairs: with bus line as keys, and the average Math scores as values.
2. A list of key/value pairs: with subway line as keys, and the average Math scores as values.

The top ten lines with highest score are shown below.

[('S1115', 612),
 ('M79', 594),
 ('Q42', 582),
 ('M22', 574),
 ('Bx3', 571),
 ('B52', 560),
 ('B63', 557),
 ('B69', 548),
 ('B54', 543),
 ('B25', 541)]